In [152]:
inpf = open('10.txt')
inp = []
for i in inpf.readlines():
    inp = inp + [i.strip()]

In [151]:
from collections import defaultdict
import scipy.signal
import numpy as np
import re
points = []

p = re.compile('^position=<\s*([-+]?\d+),\s*([-+]?\d+)> velocity=<\s*([-+]?\d+),\s*([-+]?\d+)>')


for i in inp:
    res = p.findall(i)[0]
    point = defaultdict(int)
    
    point['x'] = int(res[0])
    point['y'] = int(res[1])
    
    point['velx'] = int(res[2])
    point['vely'] = int(res[3])
    
    points.append(point)


def update(p):
    minx = 999999
    miny = 999999
    maxx = -999999
    maxy = -999999
    
    for point in p:
        point['x'] += point['velx']
        point['y'] += point['vely']
        
        minx = min(minx, point['x'])
        miny = min(miny, point['y'])
        
        maxx = max(maxx, point['x'])
        maxy = max(maxy, point['y'])
        
    return ((minx, miny), (maxx, maxy))


def create_array(p, sizeinfo):
    width = np.abs(sizeinfo[0][0] - sizeinfo[1][0])
    height = np.abs(sizeinfo[0][1] - sizeinfo[1][1])
    
    if width > 1000 or height > 1000:
        return np.zeros((1,1))
    
    arr = np.zeros((height + 1, width + 1))
    origin = sizeinfo[0]
    
    for point in p:
        arr[point['y'] - origin[1], point['x'] - origin[0]] = 1
        
    return arr


def is_text(arr):
    if (arr == 0).all():
        return False
    # check if there is at least of the points that have no neigbours at all
    res = scipy.signal.convolve2d(arr, np.ones((3,3)), mode = 'same') * arr
    ok = not (res == 1).any()
    return ok

def prettyprint(arr, i):
    print(i)
    print(arr.shape)
    for row in range(arr.shape[0]):
        s = ''
        for col in range(arr.shape[1]):
            s += '#' if arr[row, col] == 1 else ' '
        print(s)

for i in range(1, 1000000):
    sizeinfo = update(points)
    arr = create_array(points, sizeinfo)
    if is_text(arr):
        prettyprint(arr, i)
        break

10011
(10, 62)
#       #####    ####   #####   #####   #    #  ######  ######
#       #    #  #    #  #    #  #    #  #    #  #            #
#       #    #  #       #    #  #    #  #    #  #            #
#       #    #  #       #    #  #    #  #    #  #           # 
#       #####   #       #####   #####   ######  #####      #  
#       #  #    #  ###  #       #    #  #    #  #         #   
#       #   #   #    #  #       #    #  #    #  #        #    
#       #   #   #    #  #       #    #  #    #  #       #     
#       #    #  #   ##  #       #    #  #    #  #       #     
######  #    #   ### #  #       #####   #    #  ######  ######


In [113]:
print(arr.shape)

(35, 37)
